In [2]:
# import Libraries

import openai
import langchain
import pinecone 
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.llms import OpenAI

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

In [3]:
## Lets Read the document
def read_doc(directory):
    file_loader=PyPDFDirectoryLoader(directory)
    documents=file_loader.load()
    return documents

doc=read_doc('documents/')

In [5]:
## Divide the docs into chunks
### https://api.python.langchain.com/en/latest/text_splitter/langchain.text_splitter.RecursiveCharacterTextSplitter.html#
def chunk_data(docs,chunk_size=800,chunk_overlap=50):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=chunk_size,chunk_overlap=chunk_overlap)
    doc=text_splitter.split_documents(docs)
    return docs

documents=chunk_data(docs=doc)

In [4]:
## Embedding Technique Of OPENAI
embeddings=OpenAIEmbeddings(api_key=OPENAI_API_KEY)
embeddings

/Users/morganzhu/Downloads/Projects/abstract-traceback/venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x12ca2a0d0>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x12ca513d0>, model='text-embedding-ada-002', deployment='text-embedding-ada-002', openai_api_version='', openai_api_base=None, openai_api_type='', openai_proxy='', embedding_ctx_length=8191, openai_api_key='sk-proj-xwQi0dIQgkhNxaJyd0CgT3BlbkFJRyzxnzfpy9EbKT0CVOpd', openai_organization=None, allowed_special=set(), disallowed_special='all', chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None)

In [6]:
## Vector Search DB In Pinecone
from langchain_pinecone import PineconeVectorStore

index_name = "pdf-index"

pinecone = PineconeVectorStore.from_documents(
    documents, embeddings, index_name=index_name, pinecone_api_key=PINECONE_API_KEY
)

In [9]:
index_name = "pdf-index"
index=Pinecone.from_documents(doc,embeddings,index_name=index_name)

In [10]:
## Cosine Similarity Retreive Results from VectorDB
def retrieve_query(query,k=2):
    matching_results=index.similarity_search(query,k=k)
    return matching_results

In [11]:
from langchain.chains.question_answering import load_qa_chain
# from langchain import OpenAI
from langchain_openai.chat_models import ChatOpenAI

# llm=OpenAI(model_name="text-davinci-003",temperature=0.5)
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-3.5-turbo")
chain=load_qa_chain(llm,chain_type="stuff")

In [12]:
## Search answers from VectorDB
def retrieve_answers(query):
    doc_search=retrieve_query(query)
    print(doc_search)
    response=chain.run(input_documents=doc_search,question=query)
    return response

In [13]:
our_query = "简述书的内容"
answer = retrieve_answers(our_query)
print(answer)

[Document(metadata={'page': 5.0, 'source': 'documents/标杆人生.pdf'}, page_content="标杆人生  \n \n \nPage 5 同工不断地代祷与加油，与道声出版社同工的耐心配合，以及蒋海夏女士与稣文安先生在文字上\n的诸多帮忙。愿神亲自报偿。  \n一个有目的的人生旅程  \n如何善用此书  \n这不单是一本书，而是一本四十日的属灵旅 程指南。它可以帮助你找到人生最重要课题：\n“为何活在世上？ '的解答。当你完成这个旅程时，你会明白神对你的人生的目的，并且了解整\n个大局，也就是你人生的各个片段是如何拼在一起的。有了这份远眺的透视会减少你的压力、简\n化你的决择，加添满足感。最重要的，帮助你为永恒作好准备。  \n未来的四十天  \n现代人的平均寿命是二万五千五百五十天。若你在这平均值里，这就是大约你活着的时日。\n在这些日子中，分别出四十天去思考神要你在余下的光阴中做些什么事，岂非明智之举？  \n圣经清楚指出，神认为四十日是具有特别属灵意义的时段。每当神要 预备人去成就他的目的\n时，他往往使用四十天的时间：  \n挪亚的生命被四十天的雨所改变。  \n摩西在西乃山上四十天的经历改变了他的一生。  \n以色列探子四十天在应许之地，生命被改变。  \n大卫因着歌利亚四十日的挑战生命被改变。  \n以利亚仗着神的一顿饭走了四十天，生命更新而改变。  \n尼尼微城的人因着神给予的四十天机会而悔改，生命得改变。  \n主耶稣在旷野经历四十天的考验，得着力量。  \n使徒因着复活的主向他们显现并同在四十天，生命被改变。  \n在未来的四十天，你的生命也会被改变。  \n本书共分四十章，我极力敦 促你每天只读一章，这样你便有足够的时间去思想你自己的人生\n意义。圣经说： “让神改变你的思想方式，使你蜕变成为一个新造的人，你就能明白他要你做的\n事。 ” \n很多书都不能改变我们的一个原因是，我们往往操之过急，赶着追看下一章，没有停下来花\n时间认真思想所读的。我们匆促追到下个真理，而没有回想我们所学到的。  \n不要单单阅读此书，要与它互动，在重要字句下划线，在页旁写下心得，将这本书变成你自\n己的。  \n四个帮助你的特点  \n本书每章的最后一段，有一个方格子，叫做「我的人生目的省思」，在此你会

/Users/morganzhu/Downloads/Projects/abstract-traceback/venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


这本书是一本四十日的属灵旅程指南，旨在帮助读者找到人生最重要的课题：“为何活在世上？”通过完成这个旅程，读者将能够明白神对他们人生的目的，了解整个大局，并减少压力、简化决策，并为永恒做好准备。书中分为四十章，作者极力推荐每天只读一章，给予足够的时间去思考自己的人生意义。通过与书互动，划线、写下心得，读者可以将这本书变成自己的。每章末尾有一个“我的人生目的省思”部分，提供一个思考问题，浓缩了导向人生原则，可供读者在生活中反复思考。


In [14]:
from langchain_pinecone import PineconeVectorStore
from langchain_openai.chat_models import ChatOpenAI
from langchain.chains.question_answering import load_qa_chain
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

vectorstore = PineconeVectorStore(
    index_name=index_name, 
    embedding=embeddings
)
template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-3.5-turbo")
parser = StrOutputParser()

chain = (
    {"context": vectorstore.as_retriever(), "question": RunnablePassthrough()}
    | prompt
    | model
    | parser
)
chain.invoke("简述本书内容")

# query = ""
# results = vectorstore.similarity_search(query, k=1)
# response=chain.run(input_documents=results,question=query)
# print(response)

'本书是一本四十日的属灵旅程指南，旨在帮助读者找到人生最重要的课题：“为何活在世上？”完成这个旅程后，读者将明白神对他们人生的目的，并了解整个人生的大局，以及如何减少压力、简化决策，增加满足感，以及为永恒作好准备。书中引用了许多圣经中使用四十天的时段来预备人成就目的的例子，鼓励读者在未来的四十天内改变自己的生活。每章结束有“我的人生目的省思”部分，提供问题引导思考。还鼓励读者加入阅读小组，定期进行个人评估和写灵修日志，以促进灵命成长。'